### A Global Map of Artificial Intelligence Benchmark Dynamics

This page shows the interactive trajectories of averaged gain ratio for the performance increase of benchmarking datasets for NLP and CVP top hierarchies.

**Run the chunck below** to make sure all the required modules are installed and the input files are downloaded to the local running environment.

After that, run the next 4 chuncks do generate the trajectories on-the-fly.

Alternativelly press Ctrl + F9 to run all chunks! It takes 1 min to render it all.



In [1]:
#Install modules
!pip install gdown
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install -U kaleido

#Import modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplotlib
from matplotlib.ticker import ScalarFormatter
import plotly.graph_objects as go
import plotly.express as px
import re
import gdown
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#Dowload input files from my Google Drive to this local environment (files a cleared once the runtime is finished).
file_id="1l_HJTdkXNsVfGYRhoq-HgX5Altgrr5c0"
url = 'https://drive.google.com/uc?id=' + file_id
output = 'data/get_ratio_df_all_per_global_ITO_00101.csv'
gdown.download(url, output, quiet=False)
#
file_id="19uIjeF4ODFAv1RSMbG6uzpZmA6MJEw75"
url = 'https://drive.google.com/uc?id=' + file_id
output = 'data/get_ratio_df_all_per_global_ITO_00141.csv'
gdown.download(url, output, quiet=False)
#
file_id="1hVB6J4mt-RWsP1AJsqi6R30UyjzNtUuN"
url = 'https://drive.google.com/uc?id=' + file_id
output = 'data/trajectory_grouping_ITO_00101.csv'
gdown.download(url, output, quiet=False)
#
file_id="1KFd4W69i0ck1mrZCWgvjE5klp-v8M2Vm"
url = 'https://drive.google.com/uc?id=' + file_id
output = 'data/trajectory_grouping_ITO_00141.csv'
gdown.download(url, output, quiet=False)

You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ottsi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
Downloading...
From: https://drive.google.com/uc?id=1l_HJTdkXNsVfGYRhoq-HgX5Altgrr5c0
To: C:\Users\ottsi\ITO\notebooks\barbosa-silva-etal-2022\data\get_ratio_df_all_per_global_ITO_00101.csv
100%|███████████████████████████████████████████████████████████████████████████████| 714k/714k [00:00<00:00, 7.26MB/s]
Downloading...
From: https://drive.google.com/uc?id=19uIjeF4ODFAv1RSMbG6uzpZmA6MJEw75
To: C:\Users\ottsi\ITO\notebooks\barbosa-silva-etal-2022\data\get_ratio_df_all_per_global_ITO_00141.csv
100%|███████████████████████████████████████████████████████████████████████████████| 252k/252k [00:00<00:00, 5.59MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hVB6J4mt-RWsP1AJsqi6R30UyjzNtUuN
To: C:\Users\ottsi\ITO\notebooks\barbosa-silva-etal-2022\data\trajectory_grouping_ITO_00101.csv
100%|██████████████████████████████████████████████████

### Defining Plotting Function


In [67]:
#Defining Plotting Function
def plot_task_trajectory(ito, class_label, anchor):
    import pandas as pd
    #Read input
    input_file_name="data/get_ratio_df_all_per_global_"+ito+".csv"
    get_ratio_df_all_per_global = pd.read_csv(input_file_name)
    #Drop first column.
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.drop(["Unnamed: 0"], axis=1)


    #Change the column to name the facets of the plot.
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.rename(
        columns={"merge": "metricName"}
    )
    get_ratio_df_all_per_global["metricName"] = get_ratio_df_all_per_global[
        "metricName"
    ].str.replace("\\", "")

    get_ratio_df_all_per_global["unique_ds"] = (
        get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )
    get_ratio_df_all_per_global["unique_task"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )

    get_ratio_df_all_per_global["unique_task_ds_metric"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )

    def agg(ex):
      ex["ds"] = "  " + ex["ds"] + ": " + ex["metricName"] + "<BR>"
      return pd.Series({"ds": "".join(ex["ds"].sort_values().unique()), "ratio": ex["ratio"].mean()})
    
    #fix dirty data
    # get_ratio_df_all_per_global["task"] = get_ratio_df_all_per_global["task"].str.replace("Abstractive Text Summarization", "Text Summarization")
    # get_ratio_df_all_per_global["task"] = get_ratio_df_all_per_global["task"].str.replace("Document Text Summarization", "Text Summarization")
    

    #Calculate the average data frame using 'RATIO' as criteria per unique_task.
    traj = get_ratio_df_all_per_global.copy()

    #This copy is necessary to add the anchors in the plot, and avoid that they influence in the calculation of the ratio.
    traj_complete = traj.copy()
    #Select here only the anchors.
    traj_anchors = traj_complete.drop(traj_complete[traj_complete["ratio"]!=100].index)                 ######
    
    # traj_anchors.to_csv("artefacts/traj_anchors_"+ito+".csv")
    
    #This is wrong here, this is averaging just the anchor values, check the effect

    average_summary_anchors = pd.DataFrame(traj_anchors.groupby(["task", "date"])["ds", "metricName", "ratio"].apply(agg))
    #
    average_summary_anchors.sort_values(by=["date"], ascending=True)
    average_summary_anchors.reset_index(inplace=True)
    average_summary_anchors["in_trajectory"] = "OUT"
   
    #metricName causing some problems. Removed!
    traj = traj.drop(traj[traj["metricName"]=="Parameters"].index)

    #This will delete from the traj data frame all the ds/tasks which the counts are equal to 1, 
    #not forming, therefore, a trajectory.
    count_df = pd.DataFrame(traj["ds"].value_counts())
    count_df[count_df.ds == 1].index

    #Use the symbol ~ to inverselly select here.
    traj = traj[~traj["ds"].isin(count_df[count_df.ds == 1].index)]
    
    count_df = pd.DataFrame(traj["task"].value_counts())
    count_df[count_df.task == 1].index

    traj = traj[~traj["task"].isin(count_df[count_df.task == 1].index)]

    if(len(traj) == 0):
        return("Not enough points to plot trajectory")
    
    
    #We have to drop the first anchors, because we want to calculate the ratio with only the remaining meaningful points
    #traj = traj.drop(traj[traj["ratio"]==100].index) # for the individual trajectories analysis, keep these points and make them equal to Zero in the relevant notebook.
    traj = traj.drop(traj[traj["ratio"]<0].index)
    
    
    #Choose here which grouping variable to use.
    #traj = traj.drop(traj[traj["ratio"]>=100].index)                 ######  prepare average_summary , if this line is commented, then the anchors appear, but the ratio is then wrong
    traj = traj[traj["ratio"]<100].copy() 
    # traj.to_csv("artefacts/traj_"+ito+".csv")

    average_summary = pd.DataFrame(traj.groupby(["task", "date"])["ds", "metricName", "ratio"].apply(agg))
    # average_summary.to_csv("artefacts/average_summary_"+ito+".csv")
    #Drop anchors points.
    #average_summary = average_summary.drop(average_summary[average_summary["ratio"]>0.5].index)

    average_summary.sort_values(by=["date"], ascending=True)
    average_summary.reset_index(inplace=True)
    # average_summary["date"]=pd.to_datetime(average_summary['date'])
    # average_summary["date"]=average_summary["date"].dt.year
    
    

    average_summary["in_trajectory"] = 1

    i = 0
    for t in average_summary.task.unique():
        sota_per = 0
        #Here "date" can't be unique, because we want to look for the best value obtained per year.
        for v in average_summary[average_summary["task"] == t].ratio:
            per = average_summary[
                (average_summary["task"] == t) & (average_summary["ratio"] == v)
            ].ratio.astype(float)
            per = per.iloc[0]
            if per == 100:
                average_summary.loc[i, "in_trajectory"] = "OUT"
            elif per >= sota_per:
                # print(per)
                sota_per = per
                average_summary.loc[i, "in_trajectory"] = "IN"
            else:
                average_summary.loc[i, "in_trajectory"] = "IN"  # change back to OUT, if you want only to display the ratio average state of the art
                # sota_per = per
            i = i + 1

    average_summary["ratio"] = average_summary["ratio"].apply(lambda x: round(x, 4))
    
    #NOW PLOT IT
    #try using plotly
    import pandas as pd
    import plotly.express as px
    from plotly.validators.scatter.marker import SymbolValidator

    #PLACE HOLDER FOR THE ORDERING OF THE DATA FRAME #ISSUE_01
    #Use one column of this variable to sort: average_summary (get input from Kathrin)
    
    #get grouping: this tries to add the annotations by Kathrin, we need a merge here...
    grouping_table = pd.read_csv("data/trajectory_grouping_"+ito+".csv")
    
    # fix dirty data
    grouping_table["Suggested_label"] = grouping_table["Suggested_label"].str.replace("Semantic segmenation", "Semantic segmentation")
    # grouping_table["Suggested_label"] = grouping_table["Suggested_label"].str.replace("Text summarization", "Natural language generation")

    grouping_table = grouping_table.rename(columns={'Class_Label': 'task'})
    average_summary = pd.merge(average_summary, grouping_table, on = 'task', how = 'left')
    average_summary = average_summary.drop(['id', 'Superclass_id','Superclass_label'], axis = 1)
    average_summary = average_summary.dropna()
    average_summary = average_summary.sort_values('Suggested_label')
    average_summary["task"] = average_summary["Suggested_label"]+": "+average_summary["task"]

    average_summary_anchors = pd.merge(average_summary_anchors, grouping_table, on = 'task', how = 'left')
    average_summary_anchors = average_summary_anchors.drop(['id', 'Superclass_id','Superclass_label'], axis = 1)
    average_summary_anchors = average_summary_anchors.dropna()
    average_summary_anchors = average_summary_anchors.sort_values('Suggested_label')
    average_summary_anchors["task"] = average_summary_anchors["Suggested_label"]+": "+average_summary_anchors["task"]

    
    
    #split the plot for the paper
    #average_summary = average_summary.iloc[0:585]
    #average_summary = average_summary.iloc[586:len(average_summary)]

    #return(average_summary)
    
    average_summary_IN = average_summary[average_summary["in_trajectory"] == "IN"]
    
    #anchors = average_summary[average_summary["in_trajectory"] == "OUT"]
    
    #this is used to calculate the height of the plot.
    if(anchor==1):
        average_summary_OUT = average_summary_anchors.copy()
        plot_title = 'Trajectory for average gain ratio (task per year). Trajectories with single arrow shown.'
        img_file_title = "top_classes_trajectory_plots/"+ito+"_with_anchors_and_single_arrows_shown.png"
    elif(anchor==0):
        average_summary_OUT = average_summary_anchors.copy()
        plot_title = 'Trajectory for average gain ratio (task per year). Trajectories with single arrow removed.'     
        img_file_title = "top_classes_trajectory_plots/"+ito+"_with_anchors_and_single_arrows_removed.png"
    
    rows = len(average_summary_IN["task"].unique())*20
    
    #This block will take the values from average_summary_IN and delete those that have only one arrow per trajectory.
    count_df = pd.DataFrame(average_summary_IN["task"].value_counts())
    count_df[count_df.task == 1].index
    
    ##~~~~~ uncomment this block to filter out the trajectories with one arrow only
    if(anchor==0): average_summary_IN = average_summary_IN[~average_summary_IN["task"].isin(count_df[count_df.task == 1].index)]
    average_summary_IN_1_point = average_summary_IN[average_summary_IN["task"].isin(count_df[count_df.task == 1].index)]
    average_summary_IN_1_point["Single arrow"] = "Yes"
    # average_summary_IN_1_point.to_csv("average_summary_IN_1_point_"+ito+".csv", index=False)
    ##~~~~~
    #this will delete from the traj data frame, all the tasks which the average_summary_IN counts are equal to 1,
    #meaning single arrows will be excluded from the plot.


    #NEEDS to add a feature to goup based on the Suggested_label, detalhe, the ito file is fixed, change it to dynamic
    #return(average_summary_IN)

    fig_traj = px.line(average_summary_IN, 
                       x="date", 
                       y="task", 
                       color="task", 
                       )   
                                                            
    #adding anchors on both cases...
    #This function is declared to add the anchor dots (white dots) to the trajectory.
    def add_white(category,anchors_to_add,average_summary_IN,fig_traj):
        
        #select anchors that belong to selected trajectories
        anchors_to_add=anchors_to_add[anchors_to_add["task"].isin(average_summary_IN.task)].copy()
        
        fig_traj.add_trace(
            go.Scatter(
                x=anchors_to_add["date"],
                y=anchors_to_add[category],
                #facet_row="task",
                #facet_row_spacing=0.009, 
                mode="markers",
                name=None,
                marker=dict(
                    symbol=42, 
                    size=20,
                    line=dict(
                        width=2
                    ),
                    
                ),
                
                hovertemplate=
                "<BR>task: "
                + anchors_to_add[category]
                + "<BR>date: "
                + anchors_to_add["date"].astype("string")
                + "<BR>Anchor."
                + "<BR>benchmarks:<BR>"
                + anchors_to_add["ds"].astype("string"),
            )
        )

    average_summary_IN = average_summary_IN.sort_values(by="date")
    add_white("task",average_summary_anchors,average_summary_IN,fig_traj) # it seems wrong with average_summary_IN being from where the anchors are taken...
    fig_traj.add_trace(
        go.Scatter(
            x=average_summary_IN["date"],
            y=average_summary_IN["task"],
            #facet_row="task",
            #facet_row_spacing=0.009, 
            mode="markers",
            name=None,
            hovertemplate=
            "<BR>task: "
            + average_summary_IN["task"]
            + "<BR>date: "
            + average_summary_IN["date"].astype("string")
            + "<BR>ratio: "
            + average_summary_IN["ratio"].astype("string")
            + "<BR>benchmarks:<BR>"
            + average_summary_IN["ds"].astype("string"),
            marker=dict(
                size=19,  
                symbol="circle",  # https://plotly.com/python/marker-style/
                opacity=0.7,  # alpha ratio
                color=average_summary_IN["ratio"],  # set color equal to a variable
                colorscale="YlGn",  # one of plotly colorscales
                colorbar=dict(title="ratio", lenmode="pixels", len=500, thickness=10),
                showscale=True,
            ),
        )
    )

   
    
    fig_traj.update_traces(
        marker=dict(line=dict(color="black", width=1)),
        line  =dict(width=0, color="black")        
    )

    fig_traj.update_xaxes(showgrid=True, gridcolor="lightBlue", title="Year")
    #title=ito+": "+class_label
    fig_traj.update_yaxes(showgrid=True, gridcolor="lightBlue", title=None)

    # font_size 14, height=rows*1.5
    # cv, single traces --> height=rows*1.5
    fig_traj.update_layout(
        #title="Trajectory for ratio (task per year)",
        title_text=class_label,
        showlegend=False,
        font_size=21,
        plot_bgcolor="white",
        height=rows*(1.5 if (anchor and ito == "ITO_00101") else 1.35),
        width=1500,
        xaxis=dict(
            tickmode="auto",
        ),
        yaxis={'side': 'left'}
        
    )  

    
    fig_traj.update_layout(
        title={
            'y':0.995,            
            })

    fig_traj.write_html(f"artefacts/{class_label.replace(' ', '_').lower() + ('_single_arrow' if anchor else '')}.html", include_plotlyjs="cdn")
    fig_traj.write_image(f"artefacts/{class_label.replace(' ', '_').lower() + ('_single_arrow' if anchor else '')}.png", scale=2)
    fig_traj.show()
    return(average_summary_IN)

#plot_task_trajectory("ITO_00101", "Vision process", 1)


### Define get statistics function

In [68]:
#Define get statistics function
def get_statistics(traj_df):
  results=pd.DataFrame(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])

  #traj_df = traj_df_bkp.copy()

  traj_df['date'] = pd.to_datetime(traj_df['date'])
  traj_df['date'] = traj_df['date'].dt.strftime('%Y')

  for date in traj_df["date"].unique():
    
    df = traj_df[traj_df["date"]==date].copy()
    ts = df.get(['date','ratio'])
    ts['date']= pd.to_datetime(ts['date'])
    ts['date'] = ts['date'].dt.strftime('%Y-%m')
    
    
    #print(results.index)
    year = ts.describe()
    year["ratio"]=year["ratio"].astype('float').round(3)

    year = year.T
    year.index=[date]
    results=results.append(year)

  #year["ratio"].values.round(3).astype('float')\
  results = results.sort_index()
  return(results)

### Define calculate S index function

In [69]:
def calculate_S_index(results):

  data=results.T
  count = data.loc[data.index.str.contains('count')]
  mean = data.loc[data.index.str.contains('mean')]

  corrected_means=[]
  for year in count.columns:
    lambda_val = count[year].values/count.values.sum()
    corrected_means.append(mean[year].values[0]*lambda_val[0])

  S_index = sum(corrected_means) / len(corrected_means)
  return(S_index)

### Define get boxplots


In [70]:
#Define get boxplots
def get_boxplot(traj_df):
  import plotly.graph_objects as go

  traj_df['date'] = pd.to_datetime(traj_df['date'])
  traj_df['date'] = traj_df['date'].dt.strftime('%Y')

  c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, 20)] #here controls the colors


  fig = go.Figure()
  # Use x instead of y argument for horizontal plot
  i=0
  for date in traj_df["date"].unique():
    i=i+1
    df = traj_df[traj_df["date"]==date].copy()
    ts = df.get(['date','ratio'])

    ts['date']= pd.to_datetime(ts['date'])
    ts['date'] = ts['date'].dt.strftime('%Y-%m')
    #fig.add_trace(go.Box(x=ts["ratio"], name=task))
    fig.add_trace(go.Box(y=ts["ratio"], 
              boxpoints='all',
              jitter=0.8,
              whiskerwidth=0.1,
              marker_size=3, 
              line_width=2,
              name=int(date), 
              marker_color="Blue"))
    
  fig.update_layout(height=400, width=1000, showlegend=False,
                    font_size=20,
                    xaxis=dict(tickmode='linear'))
  
  fig.update_xaxes(categoryorder='array', categoryarray=list(range(2011,2021)))
                  
  return fig

## Plot the global maps for selected top hierarchies

In [77]:
anchor = 0

In [78]:
#Computer Vision Process, showing anchors and complete trajectories (single arros removed).
traj_df=plot_task_trajectory("ITO_00101", "Vision process", anchor)

boxplots=get_boxplot(traj_df)
results=get_statistics(traj_df)

#print statistics
print("Comparative yearly distribution of state-of-the-art (SOTA) averaged gain ratio values - computer vision process - single arrows removed")
boxplots.show()
results.T.style

Comparative yearly distribution of state-of-the-art (SOTA) averaged gain ratio values - computer vision process - single arrows removed


,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,2.000000,5.000000,12.000000,37.000000,79.000000,155.000000,191.000000,196.000000,21.000000
mean,0.287000,0.440000,0.406000,0.355000,0.332000,0.402000,0.427000,0.419000,0.424000
std,0.379000,0.511000,0.336000,0.287000,0.279000,0.299000,0.311000,0.342000,0.309000
min,0.019000,0.040000,0.021000,0.013000,0.000000,0.008000,0.007000,0.002000,0.010000
25%,0.153000,0.055000,0.214000,0.117000,0.098000,0.157000,0.160000,0.119000,0.208000
50%,0.287000,0.107000,0.267000,0.288000,0.278000,0.352000,0.366000,0.314000,0.420000
75%,0.421000,1.000000,0.558000,0.446000,0.499000,0.618000,0.642000,0.702000,0.591000
max,0.555000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [79]:
#Natural Language Processing, showing anchors and complete trajectories (single arrows removed).
traj_df=plot_task_trajectory("ITO_00141", "Natural Language Processing", anchor)

boxplots=get_boxplot(traj_df)
results=get_statistics(traj_df)

#print statistics
print("Comparative yearly distribution of state-of-the-art (SOTA) averaged gain ratio values - NLP - single arrows removed")
boxplots.show()
results.T.style


Comparative yearly distribution of state-of-the-art (SOTA) averaged gain ratio values - NLP - single arrows removed


,2013,2014,2015,2016,2017,2018,2019,2020
count,2.000000,8.000000,8.000000,42.000000,52.000000,82.000000,100.000000,5.000000
mean,0.555000,0.382000,0.238000,0.328000,0.325000,0.397000,0.433000,0.620000
std,0.629000,0.326000,0.150000,0.257000,0.270000,0.317000,0.286000,0.361000
min,0.110000,0.089000,0.043000,0.012000,0.017000,0.013000,0.007000,0.267000
25%,0.333000,0.196000,0.137000,0.095000,0.107000,0.151000,0.222000,0.307000
50%,0.555000,0.279000,0.224000,0.276000,0.248000,0.281000,0.403000,0.527000
75%,0.778000,0.438000,0.288000,0.506000,0.449000,0.586000,0.593000,1.000000
max,1.000000,1.000000,0.500000,0.948000,1.000000,1.000000,1.000000,1.000000


# Group level

In [80]:
#Defining Plotting Function
def plot_group_trajectory(ito, class_label, anchor):
    import pandas as pd
    #Read input
    input_file_name="data/get_ratio_df_all_per_global_"+ito+".csv"
    get_ratio_df_all_per_global = pd.read_csv(input_file_name)
    #Drop first column.
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.drop(["Unnamed: 0"], axis=1)


    #Change the column to name the facets of the plot.
    get_ratio_df_all_per_global = get_ratio_df_all_per_global.rename(
        columns={"merge": "metricName"}
    )
    get_ratio_df_all_per_global["metricName"] = get_ratio_df_all_per_global[
        "metricName"
    ].str.replace("\\", "")

    get_ratio_df_all_per_global["unique_ds"] = (
        get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )
    get_ratio_df_all_per_global["unique_task"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )

    get_ratio_df_all_per_global["unique_task_ds_metric"] = (
        get_ratio_df_all_per_global["task"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["ds"].astype(str)
        + ", "
        + get_ratio_df_all_per_global["metricName"]
    )

    def agg(ex):
      ex["ds"] = "  " + ex["subtask"] + ": " + ex["ds"] + " - " + ex["metricName"] + "<BR>"
      return pd.Series({"ds": "".join(ex["ds"].sort_values().unique()), "ratio": ex["ratio"].mean()})
    
    get_ratio_df_all_per_global["subtask"] = get_ratio_df_all_per_global["task"]
    
    #fix dirty data
    # get_ratio_df_all_per_global["task"] = get_ratio_df_all_per_global["task"].str.replace("Abstractive Text Summarization", "Text Summarization")
    # get_ratio_df_all_per_global["task"] = get_ratio_df_all_per_global["task"].str.replace("Document Text Summarization", "Text Summarization")

    #Calculate the average data frame using 'RATIO' as criteria per unique_task.
    traj = get_ratio_df_all_per_global.copy()

    #This copy is necessary to add the anchors in the plot, and avoid that they influence in the calculation of the ratio.
    traj_complete = traj.copy()
    #Select here only the anchors.
    traj_anchors = traj_complete.drop(traj_complete[traj_complete["ratio"]!=100].index)                 ######
    
    # traj_anchors.to_csv("artefacts/traj_anchors_"+ito+".csv")
    
    #This is wrong here, this is averaging just the anchor values, check the effect

    average_summary_anchors = pd.DataFrame(traj_anchors.groupby(["task", "date"])["subtask", "ds", "metricName", "ratio"].apply(agg))
    #
    average_summary_anchors.sort_values(by=["date"], ascending=True)
    average_summary_anchors.reset_index(inplace=True)
    average_summary_anchors["in_trajectory"] = "OUT"
   
    #metricName causing some problems. Removed!
    traj = traj.drop(traj[traj["metricName"]=="Parameters"].index)

    #This will delete from the traj data frame all the ds/tasks which the counts are equal to 1, 
    #not forming, therefore, a trajectory.
    count_df = pd.DataFrame(traj["ds"].value_counts())
    count_df[count_df.ds == 1].index

    #Use the symbol ~ to inverselly select here.
    traj = traj[~traj["ds"].isin(count_df[count_df.ds == 1].index)]
    
    count_df = pd.DataFrame(traj["task"].value_counts())
    count_df[count_df.task == 1].index

    traj = traj[~traj["task"].isin(count_df[count_df.task == 1].index)]

    if(len(traj) == 0):
        return("Not enough points to plot trajectory")
    
    
    #We have to drop the first anchors, because we want to calculate the ratio with only the remaining meaningful points
    #traj = traj.drop(traj[traj["ratio"]==100].index) # for the individual trajectories analysis, keep these points and make them equal to Zero in the relevant notebook.
    traj = traj.drop(traj[traj["ratio"]<0].index)
    
    
    #Choose here which grouping variable to use.
    #traj = traj.drop(traj[traj["ratio"]>=100].index)                 ######  prepare average_summary , if this line is commented, then the anchors appear, but the ratio is then wrong
    traj = traj[traj["ratio"]<100].copy() 
    # traj.to_csv("artefacts/traj_"+ito+".csv")

    average_summary = pd.DataFrame(traj.groupby(["task", "date"])["subtask", "ds", "metricName", "ratio"].apply(agg))
    # average_summary.to_csv("artefacts/average_summary_"+ito+".csv")
    #Drop anchors points.
    #average_summary = average_summary.drop(average_summary[average_summary["ratio"]>0.5].index)

    average_summary.sort_values(by=["date"], ascending=True)
    average_summary.reset_index(inplace=True)
    # average_summary["date"]=pd.to_datetime(average_summary['date'])
    # average_summary["date"]=average_summary["date"].dt.year
    
    

    average_summary["in_trajectory"] = 1

    i = 0
    for t in average_summary.task.unique():
        sota_per = 0
        #Here "date" can't be unique, because we want to look for the best value obtained per year.
        for v in average_summary[average_summary["task"] == t].ratio:
            per = average_summary[
                (average_summary["task"] == t) & (average_summary["ratio"] == v)
            ].ratio.astype(float)
            per = per.iloc[0]
            if per == 100:
                average_summary.loc[i, "in_trajectory"] = "OUT"
            elif per >= sota_per:
                # print(per)
                sota_per = per
                average_summary.loc[i, "in_trajectory"] = "IN"
            else:
                average_summary.loc[i, "in_trajectory"] = "IN"  # change back to OUT, if you want only to display the ratio average state of the art
                # sota_per = per
            i = i + 1

    average_summary["ratio"] = average_summary["ratio"].apply(lambda x: round(x, 4))
    
    #NOW PLOT IT
    #try using plotly
    import pandas as pd
    import plotly.express as px
    from plotly.validators.scatter.marker import SymbolValidator

    #PLACE HOLDER FOR THE ORDERING OF THE DATA FRAME #ISSUE_01
    #Use one column of this variable to sort: average_summary (get input from Kathrin)
    
    #get grouping: this tries to add the annotations by Kathrin, we need a merge here...
    grouping_table = pd.read_csv("data/trajectory_grouping_"+ito+".csv")
    
    # fix dirty data
    grouping_table["Suggested_label"] = grouping_table["Suggested_label"].str.replace("Semantic segmenation", "Semantic segmentation")
    # grouping_table["Suggested_label"] = grouping_table["Suggested_label"].str.replace("Text summarization", "Natural language generation")
    
    grouping_table = grouping_table.rename(columns={'Class_Label': 'task'})
    average_summary = pd.merge(average_summary, grouping_table, on = 'task', how = 'left')
    average_summary = average_summary.drop(['id', 'Superclass_id','Superclass_label'], axis = 1)
    average_summary = average_summary.dropna()
    average_summary = average_summary.sort_values('Suggested_label')
    average_summary["task"] = average_summary["Suggested_label"]+": "+average_summary["task"]

    average_summary_anchors = pd.merge(average_summary_anchors, grouping_table, on = 'task', how = 'left')
    average_summary_anchors = average_summary_anchors.drop(['id', 'Superclass_id','Superclass_label'], axis = 1)
    average_summary_anchors = average_summary_anchors.dropna()
    average_summary_anchors = average_summary_anchors.sort_values('Suggested_label')
    average_summary_anchors["task"] = average_summary_anchors["Suggested_label"]+": "+average_summary_anchors["task"]

    
    
    #split the plot for the paper
    #average_summary = average_summary.iloc[0:585]
    #average_summary = average_summary.iloc[586:len(average_summary)]

    #return(average_summary)
    
    average_summary_IN = average_summary[average_summary["in_trajectory"] == "IN"]
    
    #anchors = average_summary[average_summary["in_trajectory"] == "OUT"]
    
    #this is used to calculate the height of the plot.
    if(anchor==1):
        average_summary_OUT = average_summary_anchors.copy()
        plot_title = 'Trajectory for average gain ratio (task per year). Trajectories with single arrow shown.'
        img_file_title = "top_classes_trajectory_plots/"+ito+"_with_anchors_and_single_arrows_shown.png"
    elif(anchor==0):
        average_summary_OUT = average_summary_anchors.copy()
        plot_title = 'Trajectory for average gain ratio (task per year). Trajectories with single arrow removed.'     
        img_file_title = "top_classes_trajectory_plots/"+ito+"_with_anchors_and_single_arrows_removed.png"
    
    
    #This block will take the values from average_summary_IN and delete those that have only one arrow per trajectory.
    count_df = pd.DataFrame(average_summary_IN["task"].value_counts())
    count_df[count_df.task == 1].index
    
    ##~~~~~ uncomment this block to filter out the trajectories with one arrow only
    if(anchor==0): average_summary_IN = average_summary_IN[~average_summary_IN["task"].isin(count_df[count_df.task == 1].index)]
    average_summary_IN_1_point = average_summary_IN[average_summary_IN["task"].isin(count_df[count_df.task == 1].index)]
    average_summary_IN_1_point["Single arrow"] = "Yes"
    # average_summary_IN_1_point.to_csv("artefacts/average_summary_IN_1_point_"+ito+".csv", index=False)
    ##~~~~~
    #this will delete from the traj data frame, all the tasks which the average_summary_IN counts are equal to 1,
    #meaning single arrows will be excluded from the plot.


    #NEEDS to add a feature to goup based on the Suggested_label, detalhe, the ito file is fixed, change it to dynamic
    #return(average_summary_IN)



    average_summary_anchors=average_summary_anchors[average_summary_anchors["task"].isin(average_summary_IN.task)].copy()
    average_summary_anchors[["task", "subtask"]] = average_summary_anchors["task"].str.split(": ", 1, expand=True)
    average_summary_IN[["task", "subtask"]] = average_summary_IN["task"].str.split(": ", 1, expand=True)

    def agg_group(ex):
      return pd.Series({"ds": "".join(ex["ds"].sort_values().unique()), "ratio": ex["ratio"].mean()})
    average_summary_IN = average_summary_IN.groupby(["task", "date"])["ds", "ratio"].apply(agg_group).reset_index()
    average_summary_anchors = average_summary_anchors.groupby(["task", "date"])["ds", "ratio"].apply(agg_group).reset_index()

    if ito == "ITO_00141":
      average_summary_IN.loc[average_summary_IN["task"] == "Other NLP task", "task"] = "Text-to-image Generation"
      average_summary_anchors.loc[average_summary_anchors["task"] == "Other NLP task", "task"] = "Text-to-image Generation"

    rows = len(average_summary_IN["task"].unique())*20

    fig_traj = px.line(average_summary_IN, 
                       x="date", 
                       y="task", 
                       color="task", 
                       )
    
   
                                                            
    #adding anchors on both cases...
    #This function is declared to add the anchor dots (white dots) to the trajectory.
    def add_white(category,anchors_to_add,average_summary_IN,fig_traj):
        
        #select anchors that belong to selected trajectories
        
        fig_traj.add_trace(
            go.Scatter(
                x=anchors_to_add["date"],
                y=anchors_to_add[category],
                #facet_row="task",
                #facet_row_spacing=0.009, 
                mode="markers",
                name=None,
                marker=dict(
                    symbol=42, 
                    size=21,
                    line=dict(
                        width=2
                    ),
                    
                ),
                
                hovertemplate=
                "<BR>task: "
                + anchors_to_add[category]
                + "<BR>date: "
                + anchors_to_add["date"].astype("string")
                + "<BR>Anchor."
                + "<BR>benchmarks:<BR>"
                + anchors_to_add["ds"].astype("string"),
            )
        )

    average_summary_IN = average_summary_IN.sort_values(by="date")
    add_white("task",average_summary_anchors,average_summary_IN,fig_traj) # it seems wrong with average_summary_IN being from where the anchors are taken...
    fig_traj.add_trace(
        go.Scatter(
            x=average_summary_IN["date"],
            y=average_summary_IN["task"],
            #facet_row="task",
            #facet_row_spacing=0.009, 
            mode="markers",
            name=None,
            hovertemplate=
            "<BR>task: "
            + average_summary_IN["task"]
            + "<BR>date: "
            + average_summary_IN["date"].astype("string")
            + "<BR>ratio: "
            + average_summary_IN["ratio"].astype("string")
            + "<BR>benchmarks:<BR>"
            + average_summary_IN["ds"].astype("string"),
            marker=dict(
                size=20,  
                symbol="circle",  # https://plotly.com/python/marker-style/
                opacity=0.7,  # alpha ratio
                color=average_summary_IN["ratio"],  # set color equal to a variable
                colorscale="YlGn",  # one of plotly colorscales
                colorbar=dict(title="ratio", lenmode="pixels", len=500, thickness=10),
                showscale=True,
            ),
        )
    )

   
    
    fig_traj.update_traces(
        marker=dict(line=dict(color="black", width=1)),
        line  =dict(width=0, color="black")        
    )

    fig_traj.update_xaxes(showgrid=True, gridcolor="lightBlue", title="Year")
    # title=ito+": "+class_label
    fig_traj.update_yaxes(showgrid=True, gridcolor="lightBlue", title=None)

    
    fig_traj.update_layout(
        #title="Trajectory for ratio (task per year)",
        title_text=class_label,
        showlegend=False,
        font_size=21,
        plot_bgcolor="white",
        height=rows*2.5,
        width=1500,
        xaxis=dict(
            tickmode="auto",
        ),
        yaxis={'side': 'left'}
        
    )  

    
    fig_traj.update_layout(
        title={
            'y':0.995,            
            })

    fig_traj.show()
    fig_traj.write_html(f"artefacts/{class_label.replace(' ', '_').lower()}_grp{('_single_arrow' if anchor else '')}.html", include_plotlyjs="cdn")
    fig_traj.write_image(f"artefacts/{class_label.replace(' ', '_').lower()}_grp{('_single_arrow' if anchor else '')}.png", scale=2)
    return(average_summary_IN)

#plot_task_trajectory("ITO_00101", "Vision process", 1)


In [81]:
traj_df=plot_group_trajectory("ITO_00141", "Natural Language Processing", anchor)

In [82]:
#Computer Vision Process, showing anchors and complete trajectories (single arros removed).
traj_df=plot_group_trajectory("ITO_00101", "Vision process", anchor)